### Silver Layer

#### Table Creation

In [0]:
CREATE CATALOG IF NOT EXISTS weather;
USE CATALOG weather;
CREATE SCHEMA IF NOT EXISTS 02_silver;
USE SCHEMA 02_silver;
CREATE TABLE IF NOT EXISTS weather_api_measurements (
  -- country                     STRING,
  -- region                      STRING,
  -- district                    STRING,
  row_id                      STRING,
  location_code               STRING,
  lat                         DECIMAL(10,6),
  lon                         DECIMAL(10,6),
  tz_id                       STRING,
  api_call_timestamp          TIMESTAMP,
  last_updated_timestamp      TIMESTAMP,
  last_updated_year           INT GENERATED ALWAYS AS (YEAR(last_updated_timestamp)),
  last_updated_month          INT GENERATED ALWAYS AS (MONTH(last_updated_timestamp)),
  last_updated_day            INT GENERATED ALWAYS AS (DAY(last_updated_timestamp)),
  temp_c                      DECIMAL(10,2),
  is_day                      BOOLEAN,
  weather_condition           STRING,
  wind_mph                    DECIMAL(10,2),
  wind_degree                 DECIMAL(10,2),
  pressure_mb                 DECIMAL(10,2),
  humidity                    DECIMAL(10,2),
  cloud                       DECIMAL(10,2),
  feelslike_c                 DECIMAL(10,2),
  windchill_c                 DECIMAL(10,2),
  heatindex_c                 DECIMAL(10,2),
  dewpoint_c                  DECIMAL(10,2),
  vis_km                      DECIMAL(10,2),
  gust_mph                    DECIMAL(10,2),
  air_quality_co              DECIMAL(10,2),
  air_quality_no2             DECIMAL(10,2),
  air_quality_o3              DECIMAL(10,2),
  air_quality_so2             DECIMAL(10,2),
  air_quality_pm2_5           DECIMAL(10,2),
  air_quality_pm10            DECIMAL(10,2),
  air_quality_us_epa_index    BIGINT,
  air_quality_gb_defra_index  BIGINT,
  ingestion_timestamp         TIMESTAMP,
  processing_timestamp        TIMESTAMP,
  PRIMARY KEY (row_id),
  FOREIGN KEY (location_code) REFERENCES weather.02_silver.dim_locations(location_code)
)

TBLPROPERTIES('delta.dataSkippingNumIndexedCols' = '40');

In [0]:
WITH recent_raw AS (
  SELECT *
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY country, region, district, last_updated
                              ORDER BY ingestion_timestamp DESC) AS rn
    FROM weather.01_bronze.weather_api_measurements_raw src
    WHERE src.ingestion_timestamp > (
      SELECT COALESCE(MAX(tgt.ingestion_timestamp), TIMESTAMP '1900-01-01')
      FROM weather.02_silver.weather_api_measurements tgt
    )
  ) AS deduplicated
  WHERE rn = 1
), 

recent_raw_joined AS (
  SELECT 
    raw.*,
    dim.location_code
  FROM recent_raw AS raw
  JOIN weather.02_silver.dim_locations AS dim
  ON raw.country = dim.country
    AND raw.region = dim.region
    AND raw.district = dim.district
)

MERGE INTO weather.02_silver.weather_api_measurements AS tgt
USING recent_raw_joined AS src
ON tgt.location_code = src.location_code
   AND tgt.last_updated_timestamp = to_utc_timestamp(to_timestamp(src.last_updated, "yyyy-MM-dd HH:mm"), src.tz_id)

-- WHEN MATCHED THEN UPDATE SET
--     lat                         = CAST(src.lat AS DECIMAL(10,6)),
--     lon                         = CAST(src.lon AS DECIMAL(10,6)),
--     tz_id                       = src.tz_id,
--     api_call_timestamp          = to_utc_timestamp(to_timestamp(src.localtime, "yyyy-MM-dd HH:mm"), src.tz_id),
--     temp_c                      = CAST(src.temp_c AS DECIMAL(10,2)),
--     is_day                      = CAST(src.is_day AS BOOLEAN),
--     weather_condition           = src.condition_text,
--     wind_mph                    = CAST(src.wind_mph AS DECIMAL(10,2)),
--     wind_degree                 = CAST(src.wind_degree AS DECIMAL(10,2)),
--     pressure_mb                 = CAST(src.pressure_mb AS DECIMAL(10,2)),
--     humidity                    = CAST(src.humidity AS DECIMAL(10,2)),
--     cloud                       = CAST(src.cloud AS DECIMAL(10,2)),
--     feelslike_c                 = CAST(src.feelslike_c AS DECIMAL(10,2)),
--     windchill_c                 = CAST(src.windchill_c AS DECIMAL(10,2)),
--     heatindex_c                 = CAST(src.heatindex_c AS DECIMAL(10,2)),
--     dewpoint_c                  = CAST(src.dewpoint_c AS DECIMAL(10,2)),
--     vis_km                      = CAST(src.vis_km AS DECIMAL(10,2)),
--     gust_mph                    = CAST(src.gust_mph AS DECIMAL(10,2)),
--     air_quality_co              = CAST(src.co AS DECIMAL(10,2)),
--     air_quality_no2             = CAST(src.no2 AS DECIMAL(10,2)),
--     air_quality_o3              = CAST(src.o3 AS DECIMAL(10,2)),
--     air_quality_so2             = CAST(src.so2 AS DECIMAL(10,2)),
--     air_quality_pm2_5           = CAST(src.pm2_5 AS DECIMAL(10,2)),
--     air_quality_pm10            = CAST(src.pm10 AS DECIMAL(10,2)),
--     air_quality_us_epa_index    = src.us_epa_index,
--     air_quality_gb_defra_index  = src.gb_defra_index,
--     ingestion_timestamp         = src.ingestion_timestamp,
--     processing_timestamp        = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN INSERT (
    row_id,
    location_code,
    lat,
    lon,
    tz_id,
    api_call_timestamp,
    last_updated_timestamp,
    temp_c,
    is_day,
    weather_condition,
    wind_mph,
    wind_degree,
    pressure_mb,
    humidity,
    cloud,
    feelslike_c,
    windchill_c,
    heatindex_c,
    dewpoint_c,
    vis_km,
    gust_mph,
    air_quality_co,
    air_quality_no2,
    air_quality_o3,
    air_quality_so2,
    air_quality_pm2_5,
    air_quality_pm10,
    air_quality_us_epa_index,
    air_quality_gb_defra_index,
    ingestion_timestamp,
    processing_timestamp
)
VALUES (
    md5(concat(src.location_code, CAST(to_utc_timestamp(to_timestamp(src.last_updated, "yyyy-MM-dd HH:mm"), src.tz_id) AS STRING))),
    src.location_code,
    CAST(src.lat AS DECIMAL(10,6)),
    CAST(src.lon AS DECIMAL(10,6)),
    src.tz_id,
    to_utc_timestamp(to_timestamp(src.localtime, "yyyy-MM-dd HH:mm"), src.tz_id),
    to_utc_timestamp(to_timestamp(src.last_updated, "yyyy-MM-dd HH:mm"), src.tz_id),
    CAST(src.temp_c AS DECIMAL(10,2)),
    CAST(src.is_day AS BOOLEAN),
    src.condition_text,
    CAST(src.wind_mph AS DECIMAL(10,2)),
    CAST(src.wind_degree AS DECIMAL(10,2)),
    CAST(src.pressure_mb AS DECIMAL(10,2)),
    CAST(src.humidity AS DECIMAL(10,2)),
    CAST(src.cloud AS DECIMAL(10,2)),
    CAST(src.feelslike_c AS DECIMAL(10,2)),
    CAST(src.windchill_c AS DECIMAL(10,2)),
    CAST(src.heatindex_c AS DECIMAL(10,2)),
    CAST(src.dewpoint_c AS DECIMAL(10,2)),
    CAST(src.vis_km AS DECIMAL(10,2)),
    CAST(src.gust_mph AS DECIMAL(10,2)),
    CAST(src.co AS DECIMAL(10,2)),
    CAST(src.no2 AS DECIMAL(10,2)),
    CAST(src.o3 AS DECIMAL(10,2)),
    CAST(src.so2 AS DECIMAL(10,2)),
    CAST(src.pm2_5 AS DECIMAL(10,2)),
    CAST(src.pm10 AS DECIMAL(10,2)),
    src.us_epa_index,
    src.gb_defra_index,
    src.ingestion_timestamp,
    CURRENT_TIMESTAMP()
);


In [0]:
OPTIMIZE weather.02_silver.weather_api_measurements ZORDER BY last_updated_timestamp